In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp


In [2]:
# review ufo df
ufo = pd.read_csv('../data/ufo_clean.csv', index_col = 0)
ufo = ufo.rename(columns={'index':'ufo_id_num'})
ufo['ufo_id_num'] += 1
ufo.head(1)

,ufo_id_num,year_week,date_of_sight,year,month,week_of_year,state,city,sight_summary,ufo_shape
0,1,2006-52,2006-01-01,2006,1,52,NY,I-80 (unknown city proximity),"Traveling from Brooklyn, NY to Groton, NY on I...",Formation


In [6]:
# empty storages for google df data
googledfs = []
googledfnames = []
googledfdict = dict(zip(googledfnames,googledfs))

In [7]:
# trump import, rename, sorting, assign topic
trump = pd.read_csv('../Data/Raw/Trump.csv',header=1).sort_values('Week').rename(columns={'Donald Trump: (United States)':'rel_val'})
trump['topic']="trump"
googledfs.append(trump)
googledfnames.append('trump')
trump.head(1)

,Week,rel_val,topic
0,2016-05-01,14,trump


In [8]:
# obama import, rename, sorting, assign topic
obama = pd.read_csv('../Data/Raw/Obama.csv',header=1).sort_values('Week').rename(columns={'Barack Obama: (United States)':'rel_val'})
obama['topic']="obama"
googledfs.append(obama)
googledfnames.append('obama')
obama.head(1)

,Week,rel_val,topic
0,2008-05-04,9,obama


In [213]:
# kremlin  import, rename, sorting, assign topic
kremlin =  pd.read_csv('../Data/Raw/Kremlin.csv',header=1).sort_values('Week').rename(columns={'Kremlin: (United States)':'rel_val'})
kremlin['topic']="kremlin"
googledfs.append(kremlin)
googledfnames.append('kremlin')
kremlin.head(1)

,Week,rel_val,topic
0,2014-01-05,0,kremlin


In [9]:
# ebola import, rename, sorting, assign topic
ebola =  pd.read_csv('../Data/Raw/Ebola.csv',header=1).sort_values('Week').rename(columns={'Ebola: (United States)':'rel_val'})
ebola['topic']="ebola"
googledfs.append(ebola)
googledfnames.append('ebola')
ebola.head(1)

,Week,rel_val,topic
0,2014-04-06,1,ebola


In [10]:
# corona import, rename, sorting, assign topic
corona =  pd.read_csv('../Data/Raw/Corona.csv',header=1).sort_values('Week').rename(columns={'Coronavirus: (United States)':'rel_val'})
corona['topic']="corona"
googledfnames.append('corona')
googledfs.append(corona)
corona.head(1)

,Week,rel_val,topic
0,2019-09-01,<1,corona


In [11]:
# assign datetime col to all dfs
for df in googledfs:
    df['week_dt'] = pd.to_datetime(df['Week'])

In [12]:
# confirm
corona.head(1)

,Week,rel_val,topic,week_dt
0,2019-09-01,<1,corona,2019-09-01


In [13]:
# concat all dfs to "alldfs"
alldfs = pd.concat(googledfs).drop(columns='Week')

In [14]:
# assign cols "year","month","week_of_year"
alldfs['year'] = alldfs['week_dt'].dt.year.astype(str)
alldfs['month'] = alldfs['week_dt'].dt.month.astype(str)
alldfs['week_of_year'] = alldfs['week_dt'].dt.weekofyear

<ipython-input-14-300f796a80b8>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  alldfs['week_of_year'] = alldfs['week_dt'].dt.weekofyear


In [15]:
#review
alldfs.head()

,rel_val,topic,week_dt,year,month,week_of_year
0,14,trump,2016-05-01,2016,5,17
1,9,trump,2016-05-08,2016,5,18
2,8,trump,2016-05-15,2016,5,19
3,8,trump,2016-05-22,2016,5,20
4,8,trump,2016-05-29,2016,5,21


In [233]:
# assign "week_id" col
alldfs['week_id'] = alldfs['year'] + "-" + alldfs['week_of_year'].astype(str)

In [234]:
# confirm
alldfs.head()

,rel_val,topic,week_dt,year,month,week_of_year,week_id
0,14,trump,2016-05-01,2016,5,17,2016-17
1,9,trump,2016-05-08,2016,5,18,2016-18
2,8,trump,2016-05-15,2016,5,19,2016-19
3,8,trump,2016-05-22,2016,5,20,2016-20
4,8,trump,2016-05-29,2016,5,21,2016-21


In [16]:
# export to csv, set to comment after export:
# alldfs.to_csv("../Data/Datasets/googletopics_weekid.csv")